# Test Sudokus for a Unique Solution

In [56]:
import sys
sys.path.append("..")
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType 
from pyspark.sql.functions import udf, struct
import numpy as np
from data_conversions import convert_to_block
from SudokuSolver import Sudoku
from helpers.data_prep_and_print import print_df

## Create Spark Session

In [57]:
spark = (SparkSession
       .builder
       .appName("Sudoku Single Solustion Exists")
       .getOrCreate())
spark.sparkContext.setLogLevel("ERROR")

## Create a DataFrame using an ifered Schema 

In [58]:
path = "../sudoku_data/"
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .csv(path+"sudoku_1_mio_sequenced_columns.csv")
df = df.limit(10)
#print_df (df)

In [59]:
#df.foreach(lambda x: 
#    print("Difficulty: " + Sudoku(convert_to_block(x)).evaluate()) 
    
#)

In [60]:
# add the difficulty label to the data frame 
difficulty_calc = udf(lambda x: Sudoku(convert_to_block(x)).evaluate(), StringType())
df_with_difficulty = df.withColumn("difficulty", difficulty_calc(struct(*list(df.columns))))
print_df(df_with_difficulty,10)
# df_with_difficulty.write.csv(path+"sudoku_1_mio_sequenced_columns_with_difficulty.csv",header=True, mode="overwrite")

,Q1A,Q1B,Q1C,Q1D,Q1E,Q1F,Q1G,Q1H,Q1I,Q2A,Q2B,Q2C,Q2D,Q2E,Q2F,Q2G,Q2H,Q2I,Q3A,Q3B,Q3C,Q3D,Q3E,Q3F,Q3G,Q3H,Q3I,Q4A,Q4B,Q4C,Q4D,Q4E,Q4F,Q4G,Q4H,Q4I,Q5A,Q5B,Q5C,Q5D,Q5E,Q5F,Q5G,Q5H,Q5I,Q6A,Q6B,Q6C,Q6D,Q6E,Q6F,Q6G,Q6H,Q6I,Q7A,Q7B,Q7C,Q7D,Q7E,Q7F,Q7G,Q7H,Q7I,Q8A,Q8B,Q8C,Q8D,Q8E,Q8F,Q8G,Q8H,Q8I,Q9A,Q9B,Q9C,Q9D,Q9E,Q9F,Q9G,Q9H,Q9I,S1A,S1B,S1C,S1D,S1E,S1F,S1G,S1H,S1I,S2A,S2B,S2C,S2D,S2E,S2F,S2G,S2H,S2I,S3A,S3B,S3C,S3D,S3E,S3F,S3G,S3H,S3I,S4A,S4B,S4C,S4D,S4E,S4F,S4G,S4H,S4I,S5A,S5B,S5C,S5D,S5E,S5F,S5G,S5H,S5I,S6A,S6B,S6C,S6D,S6E,S6F,S6G,S6H,S6I,S7A,S7B,S7C,S7D,S7E,S7F,S7G,S7H,S7I,S8A,S8B,S8C,S8D,S8E,S8F,S8G,S8H,S8I,S9A,S9B,S9C,S9D,S9E,S9F,S9G,S9H,S9I,difficulty
0,0,0,4,3,0,0,2,0,9,0,0,5,0,0,9,0,0,1,0,7,0,0,6,0,0,4,3,0,0,6,0,0,2,0,8,7,1,9,0,0,0,7,4,0,0,0,5,0,0,8,3,0,0,0,6,0,0,0,0,0,1,0,5,0,0,3,5,0,8,6,9,0,0,4,2,9,1,0,3,0,0,8,6,4,3,7,1,2,5,9,3,2,5,8,4,9,7,6,1,9,7,1,2,6,5,8,4,3,4,3,6,1,9,2,5,8,7,1,9,8,6,5,7,4,3,2,2,5,7,4,8,3,9,1,6,6,8,9,7,3,4,1,2,5,7,1,3,5,2,8,6,9,4,5,4,2,9,1,6,3,7,8,Medium
1,0,4,0,1,0,0,0,5,0,1,0,7,0,0,3,9,6,0,5,2,0,0,0,8,0,0,0,0,0,0,0,0,0,0,1,7,0,0,0,9,0,6,8,0,0,8,0,3,0,5,0,6,2,0,0,9,0,0,6,0,5,4,3,6,0,0,0,8,0,7,0,0,2,5,0,0,9,7,1,0,0,3,4,6,1,7,9,2,5,8,1,8,7,5,2,3,9,6,4,5,2,9,6,4,8,3,7,1,9,6,5,8,3,2,4,1,7,4,7,2,9,1,6,8,3,5,8,1,3,7,5,4,6,2,9,7,9,8,2,6,1,5,4,3,6,3,1,4,8,5,7,9,2,2,5,4,3,9,7,1,8,6,Medium
2,6,0,0,1,2,0,3,8,4,0,0,8,4,5,9,0,7,2,0,0,0,0,0,6,0,0,5,0,0,0,2,6,4,0,3,0,0,7,0,0,8,0,0,0,6,9,4,0,0,0,3,0,0,0,3,1,0,0,0,0,0,5,0,0,8,9,7,0,0,0,0,0,5,0,2,0,0,0,1,9,0,6,9,5,1,2,7,3,8,4,1,3,8,4,5,9,6,7,2,7,2,4,8,3,6,9,1,5,8,5,1,2,6,4,7,3,9,2,7,3,9,8,1,5,4,6,9,4,6,5,7,3,8,2,1,3,1,7,6,9,2,4,5,8,4,8,9,7,1,5,2,6,3,5,6,2,3,4,8,1,9,7,Medium
3,4,9,7,2,0,0,0,0,0,1,0,0,4,0,0,0,0,5,0,0,0,0,1,6,0,9,8,6,2,0,3,0,0,0,4,0,3,0,0,9,0,0,0,0,0,0,0,1,0,7,2,6,0,0,0,0,2,0,0,5,8,7,0,0,0,0,6,0,0,0,0,4,5,3,0,0,9,7,0,6,1,4,9,7,2,5,8,3,1,6,1,8,6,4,3,9,7,2,5,2,5,3,7,1,6,4,9,8,6,2,9,3,8,1,5,4,7,3,7,5,9,6,4,1,8,2,8,4,1,5,7,2,6,3,9,9,6,2,1,4,5,8,7,3,7,1,8,6,2,3,9,5,4,5,3,4,8,9,7,2,6,1,Medium
4,0,0,5,9,1,0,3,0,8,0,0,9,4,0,3,0,6,0,0,2,7,5,0,0,1,0,0,0,3,0,0,0,0,2,0,1,0,0,0,8,2,0,0,0,7,0,0,6,0,0,7,0,0,4,0,0,0,0,8,0,0,0,0,6,4,0,1,5,0,7,0,0,8,9,0,0,0,0,4,2,0,4,6,5,9,1,2,3,7,8,1,8,9,4,7,3,5,6,2,3,2,7,5,6,8,1,4,9,7,3,8,6,4,5,2,9,1,9,5,4,8,2,1,6,3,7,2,1,6,3,9,7,8,5,4,5,7,3,2,8,4,9,1,6,6,4,2,1,5,9,7,8,3,8,9,1,7,3,6,4,2,5,Medium
5,1,0,0,0,0,5,0,0,7,3,8,0,9,0,0,0,0,0,6,0,0,0,0,0,4,8,0,8,2,0,0,0,1,0,7,5,0,4,0,7,6,0,0,2,0,0,6,9,0,0,2,0,0,1,0,0,5,0,3,9,0,0,4,0,0,0,0,2,0,1,0,0,0,0,0,0,4,6,3,5,2,1,9,4,6,8,5,2,3,7,3,8,2,9,7,4,5,1,6,6,5,7,2,1,3,4,8,9,8,2,3,4,9,1,6,7,5,5,4,1,7,6,8,9,2,3,7,6,9,3,5,2,8,4,1,2,1,5,8,3,9,7,6,4,4,3,6,5,2,7,1,9,8,9,7,8,1,4,6,3,5,2,Medium
6,0,0,9,0,6,5,4,3,0,0,0,7,0,0,0,8,0,0,6,0,0,1,0,8,0,2,0,0,0,3,0,9,0,0,0,2,5,0,1,4,0,3,9,6,0,8,0,4,0,0,0,1,0,0,0,3,0,5,0,9,0,0,7,0,5,6,0,8,0,0,0,0,0,7,0,2,4,0,0,9,0,2,8,9,7,6,5,4,3,1,3,1,7,9,2,4,8,5,6,6,4,5,1,3,8,7,2,9,7,6,3,8,9,1,5,4,2,5,2,1,4,7,3,9,6,8,8,9,4,6,5,2,1,7,3,4,3,2,5,1,9,6,8,7,9,5,6,3,8,7,2,1,4,1,7,8,2,4,6,3,9,5,Medium
7,0,0,0,0,0,0,6,5,7,7,0,2,4,0,0,1,0,0,3,5,0,0,0,6,0,0,0,5,0,0,0,2,0,0,0,9,2,1,0,3,0,0,5,0,0,0,4,7,1,0,9,0,0,8,0,0,8,7,6,0,0,9,0,9,0,0,5,0,2,0,3,0,0,3,0,0,1,8,2,0,6,8,9,4,2,3,1,6,5,7,7,6,2,4,9,5,1,8,3,3,5,1,8,7,6,9,4,2,5,8,3,6,2,4,7,1,9,2,1,9,3,8,7,5,6,4,6,4,7,1,5,9,3,2,8,1,2,8,7,6,3,4,9,5,9,7,6,5,4,2,8,3,1,4,3,5,9,1,8,2,7,6,Medium
8,5,0,3,0,7,0,1,9,0,0,0,0,0,0,6,7,5,0,0,4,7,1,9,0,6,0,0,4,0,0,0,3,8,0,0,0,9,5,0,2,0,0,3,0,0,0,0,0,0,1,0,0,7,2,0,0,0,8,0,4,0,0,1,3,0,0,0,0,1,8,6,0,0,8,6,7,2,0,0,0,5,5,6,3,4,7,2,1,9,8,2,1,9,3,8,6,7,5,4,8,4,7,1,9,5,6,2,3,4,7,2,6,3,8,5,1,9,9,5,1,2,4,7,3,8,6,6,3,8,5,1,9,4,7,2,7,9,5,8,6,4,2,3,1,3,2,4,9,5,1,8,6,7,1,8,6,7,2,3,9,4,5,Medium
9,0,6,0,7,2,0,9,0,8,0,8,4,0,0,3,0,0,1,7,0,0,1,0,0,0,6,5,9,0,0,0,0,8,0,0,0,0,7,1,0,6,0,0,0,0,0,0,2,0,1,0,0,3,4,0,0,0,2,0,0,7,0,6,0,3,0,0,4,9,8,0,0,2,1,5,0,0,0,0,9,0,1,6,3,7,2,5,9,4,8,5,8,4,6,9,3,2,7,1,7,2,9,1,8,4,3,6,5,9,4,6,3,5,8,1,2,7,3,7,1,4,6,2,5,8,9,8,5,2,9,1,7,6,3,4,4,9,8,2,3,1,7,5,6,6,3,7,5,4,9,8,1,2,2,1,5,8,7,6,4,9,3,Medium


## Test if a single solution exists

In [61]:
#df.foreach(lambda x:  
#           print(f"Solutions  {len(list(Sudoku(convert_to_block(x)).solve()))}")
#           )
num_solutions_calc = udf(lambda x: len(list(Sudoku(convert_to_block(x)).solve())), IntegerType())
df_with_only_one_solution = df_with_difficulty.withColumn("num_solutions", num_solutions_calc(struct(*list(df.columns))))
print_df(df_with_only_one_solution,10)
#df_with_only_one_solution.write.csv(path+"sudoku_1_mio_sequenced_columns_with_difficulty_and_num_solutions.csv",header=True, mode="overwrite")